In [1]:
using CairoMakie;
using DelimitedFiles, StatsBase, Statistics, LazySets, Glob, LaTeXStrings, Printf, NLsolve;
include("analysis/3spec_analysis_library.jl")

norm_diff (generic function with 1 method)

In [ ]:
nu = 0.001
N = 200
as = 6
ls = 0.0009
xstep = 0.025
xstep_dr = 0.03

tini = 75

fastring = "fa1.01*"

listdir = glob("data/*"*"nu$(nu)*"*fastring)

land_vec = readdlm(pwd()*"/"*listdir[1]*"/landvec.txt")
avg_vec = copy(transpose(mean(land_vec, dims=1)))
var_vec = copy(transpose(var(land_vec, dims=1)))

# plot the landscapes
f = Figure(backgroundcolor = RGBf(1,1,1), size = (800, 500))
gde = f[1, 1] = GridLayout()
gdr = gde[1,1] = GridLayout()
gdi = gde[1,2] = GridLayout()
g2 = f[2,1] = GridLayout()
g2f = g2[1,1] = GridLayout()
g2V = g2[1,2] = GridLayout()
adr = Axis(gdr[1, 1], xgridvisible = false, ygridvisible = false)
adi = Axis(gdi[1, 1], xgridvisible = false, ygridvisible = false)
#hideydecorations!(adi,ticklabels = true, ticks=false)
[ad.xlabel = L"\bar{f}(z) - f_c" for ad in [adr, adi]]
adr.ylabel = L"\bar{V}(z) - V_c" 
adr.title = "drift (mini-b.)"
adi.title = "diffusion (mini-b.)"
max = maximum(land_vec)
formatted_fa = @sprintf("%.2f", avg_vec[1,1])
formatted_fb = @sprintf("%.2f", avg_vec[2,1])
formatted_fc = @sprintf("%.2f", avg_vec[3,1])
formatted_Va = @sprintf("%.2f", var_vec[1,1])
formatted_Vb = @sprintf("%.2f", var_vec[2,1])
formatted_Vc = @sprintf("%.2f", var_vec[3,1])

slope1 = (var_vec[2,1]-var_vec[3,1])/(avg_vec[2,1]-avg_vec[3,1])
slope2 = (var_vec[1,1]-var_vec[3,1])/(avg_vec[1,1]-avg_vec[3,1])
points = reduce(vcat,[[Point2f(x, y) for y in range(slope1*x,slope2*x,minimum([10,Int(floor((slope2-slope1)*x/0.015))]))] for x in 0:xstep_dr:maximum(avg_vec)-avg_vec[3,1]])

#drift 
scatter!(adr,mean(land_vec,dims=1)[1,:].-avg_vec[3,1],var(land_vec,dims=1)[1,:].-var_vec[3,1],markersize=5,color=:black)
lines!(adr,push!(avg_vec[:,1],avg_vec[1,1]).-avg_vec[3,1],push!(var_vec[:,1],var_vec[1,1]).-var_vec[3,1],color=:black,linewidth=0.5)
v = [drift_pt(x) for x in points]
strength = [norm(v[i]) for i in 1:length(v)]
a1 = arrows!(adr,points,v, arrowsize = as, lengthscale = ls, arrowcolor = strength, linecolor = strength, colormap = cgrad(:plasma))
text!(adr,0.31,1.05,text=L"$\mathbf{\bar{f}}$",color=Makie.wong_colors()[6])
text!(adr,0.3,0.85,text=L"\mathbf{\bar{V}}",color=Makie.wong_colors()[3]) 

Colorbar(gdr[1,2],a1)

[ylims!(ad,-0.72,1.25) for ad in [adr,adi]]

#inset evo
N = 200
ncopies = 50

dir = glob("data/N$(N)_*"*fastring)[1]
F = []
V = []

for n in 1:ncopies
    pop = readdlm(pwd()*"/"*dir*"/balanced/copy_$(n)/pop.txt")[:,2:end]./N
    favg = pop*avg_vec
    vavg = pop*var_vec
    append!(F,favg[:,1])
    append!(V,vavg[:,1])
end

bboxf = lift(adr.scene.camera.projectionview, adr.scene.px_area) do _, pxa
    p = Makie.project(adr.scene, Point(0.28, 1.2))
    c = p + pxa.origin
    Rect2f(c .- Point2f(100, 50), (100, 50))
end
axinf = Axis(f, bbox = bboxf,xgridvisible = false,ygridvisible = false)

Fres = reshape(F.-avg_vec[3,1], Int(length(F)/ncopies),ncopies)
lines!(axinf,0:20:5000,mean(Fres,dims=2)[:,1],linewidth=3,color=Makie.wong_colors()[6])
band!(axinf,0:20:5000,mean(Fres,dims=2)[:,1].-std(Fres,dims=2)[:,1]./sqrt(ncopies),mean(Fres,dims=2)[:,1].+std(Fres,dims=2)[:,1]./sqrt(ncopies),
    color=Makie.wong_colors()[6],alpha=0.5)
Vres = reshape(V.-var_vec[3,1], Int(length(V)/ncopies),ncopies)
lines!(axinf,0:20:5000,mean(Vres,dims=2)[:,1],linewidth=3,color=Makie.wong_colors()[3])
band!(axinf,0:20:5000,mean(Vres,dims=2)[:,1].-std(Vres,dims=2)[:,1]./sqrt(ncopies),mean(Vres,dims=2)[:,1].+std(Vres,dims=2)[:,1]./sqrt(ncopies),
    color=Makie.wong_colors()[3],alpha=0.5)
axinf.xlabel="gen."
tight_xticklabel_spacing!(axinf)

#diffusion
scatter!(adi,mean(land_vec,dims=1)[1,:].-avg_vec[3,1],var(land_vec,dims=1)[1,:].-var_vec[3,1],markersize=5,color=:black)
lines!(adi,push!(avg_vec[:,1],avg_vec[1,1]).-avg_vec[3,1],push!(var_vec[:,1],var_vec[1,1]).-var_vec[3,1],color=:black,linewidth=0.5)

points_diff = reduce(vcat,[[Point2f(x, y) for y in range(slope1*x,slope2*x,maximum([1,minimum([15,Int(floor((slope2-slope1)*x/0.03))])]))] for x in 0.025:xstep:maximum(avg_vec)-avg_vec[3,1]])
spmax = [diffmax_pt(x) for x in points_diff]
strength_max = [norm(spmax[i]) for i in 1:length(spmax)]
spmin = [diffmin_pt(x) for x in points_diff]
strength_min = [norm(spmin[i]) for i in 1:length(spmin)]

spmax0 = [diffmax0_pt(x) for x in points_diff]
strength_max0 = [norm(spmax0[i]) for i in 1:length(spmax0)]
spmin0 = [diffmin0_pt(x) for x in points_diff]
strength_min0 = [norm(spmin0[i]) for i in 1:length(spmin0)]

cmin = minimum([minimum(strength_min), minimum(strength_min0)])
cmax = maximum([maximum(strength_max), maximum(strength_max0)])
a2 = arrows!(adi,points_diff,spmax, arrowsize = 0, lengthscale = 0.07, arrowcolor = strength_max, linecolor = strength_max, 
    colormap = cgrad(:plasma),linewidth=2, align=:origin, colorrange = (cmin, cmax))
arrows!(adi,points_diff,spmin, arrowsize = 0, lengthscale = 0.07, arrowcolor = strength_min, linecolor = strength_min, 
    colormap = cgrad(:plasma),linewidth=1, align = :center, colorrange = (cmin, cmax))

Colorbar(gdi[1,2],a2)

bbox = lift(adi.scene.camera.projectionview, adi.scene.px_area) do _, pxa
    p = Makie.project(adi.scene, Point(0.2, 1.15))
    c = p + pxa.origin
    Rect2f(c .- Point2f(85, 75), (85, 75))
end

text!(adi,-0.04,0.7,text=L"$\mathbf{c}$")
text!(adi,0.2,0.42,text=L"$\mathbf{b}$")
text!(adi,0.2,1.12,text=L"$\mathbf{a}$")

axin = Axis(f, bbox = bbox,xgridvisible = false,ygridvisible = false)
hidedecorations!(axin)
text!(adi,0.01,1.05,text="(avg)",font=:bold)
hidespines!(axin)
scatter!(axin,mean(land_vec,dims=1)[1,:].-avg_vec[3,1],var(land_vec,dims=1)[1,:].-var_vec[3,1],markersize=5,color=:black)
lines!(axin,push!(avg_vec[:,1],avg_vec[1,1]).-avg_vec[3,1],push!(var_vec[:,1],var_vec[1,1]).-var_vec[3,1],color=:black,linewidth=0.5)

arrows!(axin,points_diff,spmax0, arrowsize = 0, lengthscale = 0.15, arrowcolor = strength_max0, linecolor = strength_max0, 
    colormap = cgrad(:plasma),linewidth=1.5, align=:origin, colorrange = (cmin, cmax))
arrows!(axin,points_diff,spmin0, arrowsize = 0, lengthscale = 0.15, arrowcolor = strength_min0, linecolor = strength_min0, 
    colormap = cgrad(:plasma),linewidth=0.75, align = :center, colorrange = (cmin, cmax))

sol = fixed(avg_vec,var_vec,nu,N)
sol0 = fixed0(avg_vec,var_vec,nu)

P = copy(sol.zero)
P0 = copy(sol0.zero)

scatter!(adr,P[1],P[2],markersize=10,color=Makie.wong_colors()[1],strokecolor=:black, strokewidth=0.8)
scatter!(adi,P[1],P[2],markersize=10,color=Makie.wong_colors()[1],strokecolor=:black, strokewidth=0.8)
scatter!(adr,P0[1],P0[2],markersize=10,color=Makie.wong_colors()[2],strokecolor=:black, strokewidth=0.8)
scatter!(adi,P0[1],P0[2],markersize=10,color=Makie.wong_colors()[2],strokecolor=:black, strokewidth=0.8)

for ad in [adr,adi]
    text!(ad,-0.035,0.01,text=L"$\mathbf{c}$",align=(:center,:center))
    text!(ad,0.56,-0.65,text=L"$\mathbf{b}$",align=(:center,:center))
    text!(ad,0.57,1.06,text=L"$\mathbf{a}$",align=(:center,:center))
end


################################

#phase diagrams
Narr = [50,100,200,300,500,1000,2000,5000]

trans_f = [Axis(g2f[1, i], xgridvisible = false, ygridvisible = false) for i in 1:2]
trans_V = [Axis(g2V[1, i], xgridvisible = false, ygridvisible = false) for i in 1:2]

nbins = 40
ms = 10
listdir = [glob("data/N$(N)_*"*fastring) for N in Narr]

meanf = []
meanv = []
meanf_ref = []
meanv_ref = []

fp_fit = []
fp_ref_fit = []
fp_var = []
fp_ref_var = []

mfit = []
mvar = []
mfit_ref = []   
mvar_ref = []

fmin = 0.39
for dir in listdir
    F = []
    V = []
    Fref = []
    Vref = []
    N = parse(Int,split(split(dir[1],"N")[2],"_")[1])
    for n in 1:ncopies
        pop = readdlm(pwd()*"/"*dir[1]*"/balanced/copy_$(n)/pop.txt")[tini:end,2:end]./N
        pop_ref = readdlm(pwd()*"/"*dir[1]*"/balanced/copy_$(n)/pop_ref.txt")[tini:end,2:end]./N
        favg = pop*avg_vec
        favg_ref = pop_ref*avg_vec
        vavg = pop*var_vec
        vavg_ref = pop_ref*var_vec   
        append!(F,favg[:,1])
        append!(Fref,favg_ref[:,1])
        append!(V,vavg[:,1])
        append!(Vref,vavg_ref[:,1])
    end
    fhist = fit(Histogram,(F.-avg_vec[3,1]),range(fmin-0.001,avg_vec[1,1]-avg_vec[3,1]+0.001,nbins))
    fref_hist = fit(Histogram,(Fref.-avg_vec[3,1]),range(fmin-0.001,avg_vec[1,1]-avg_vec[3,1]+0.001,nbins))

    Vhist = fit(Histogram,(V.-var_vec[3,1]),range(var_vec[2,1]-var_vec[3,1]-0.001,var_vec[1,1]-var_vec[3,1]+0.001,nbins))
    Vref_hist = fit(Histogram,(Vref.-var_vec[3,1]),range(var_vec[2,1]-var_vec[3,1]-0.001,var_vec[1,1]-var_vec[3,1]+0.001,nbins))

    push!(meanf,fhist.weights)
    push!(meanv,Vhist.weights)
    push!(meanf_ref,fref_hist.weights)
    push!(meanv_ref,Vref_hist.weights)
    
    sol = fixed(avg_vec,var_vec,nu,N)
    sol0 = fixed0(avg_vec,var_vec,nu)
    push!(fp_fit, sol.zero[1])
    push!(fp_ref_fit, sol0.zero[1])
    push!(fp_var, sol.zero[2])
    push!(fp_ref_var, sol0.zero[2])
    push!(mfit, mean(F.-avg_vec[3,1]))
    push!(mvar, mean(V.-var_vec[3,1]))
    push!(mfit_ref, mean(Fref.-avg_vec[3,1]))
    push!(mvar_ref, mean(Vref.-var_vec[3,1]))
end

hideydecorations!(trans_f[2],ticks=false,ticklabels=true)
hideydecorations!(trans_V[2],ticks=false,ticklabels=true)
logdistr = true
if logdistr
    meanf = log.(reduce(hcat,meanf))
    meanf_ref = log.(reduce(hcat,meanf_ref))
    meanv = log.(reduce(hcat,meanv))
    meanv_ref = log.(reduce(hcat,meanv_ref))

    meanf[meanf .== -Inf] .= NaN
    meanf_ref[meanf_ref .== -Inf] .= NaN
    meanv[meanv .== -Inf] .= NaN
    meanv_ref[meanv_ref .== -Inf] .= NaN
else
    meanf = reduce(hcat,meanf)
    meanf_ref = reduce(hcat,meanf_ref)
    meanv = reduce(hcat,meanv)
    meanv_ref = reduce(hcat,meanv_ref)

end


heatmap!(trans_f[1],range(fmin-0.001,avg_vec[1,1]-avg_vec[3,1]+0.001,nbins),log2.(Narr),meanf,colormap = :BuPu)
cb1 = heatmap!(trans_f[2],range(fmin-0.001,avg_vec[1,1]-avg_vec[3,1]+0.001,nbins),log2.(Narr),meanf_ref,colormap = :BuPu)
heatmap!(trans_V[1],range(var_vec[2,1]-var_vec[3,1]-0.001,var_vec[1,1]-var_vec[3,1]+0.001,nbins),log2.(Narr),meanv,colormap=:BuPu)
cb2 = heatmap!(trans_V[2],range(var_vec[2,1]-var_vec[3,1]-0.001,var_vec[1,1]-var_vec[3,1]+0.001,nbins),log2.(Narr),meanv_ref,colormap=:BuPu)

lines!(trans_f[1],fp_ref_fit,log2.(Narr),color=Makie.wong_colors()[2],label = "FP (avg)")
#lines!(trans_f[1],mfit_ref,log2.(Narr),color=Makie.wong_colors()[2],linestyle=:dash,label = "mean")
lines!(trans_f[1],fp_fit,log2.(Narr),color=Makie.wong_colors()[1],label = "FP (m-b)")
#lines!(trans_f[1],mfit,log2.(Narr),color=Makie.wong_colors()[1],linestyle=:dash,label = "mean ")


lines!(trans_V[1],fp_ref_var,log2.(Narr),color=Makie.wong_colors()[2])
#lines!(trans_V[1],mvar_ref,log2.(Narr),color=Makie.wong_colors()[2],linestyle=:dash)
lines!(trans_V[1],fp_var,log2.(Narr),color=Makie.wong_colors()[1])
#lines!(trans_V[1],mvar,log2.(Narr),color=Makie.wong_colors()[1],linestyle=:dash)


lines!(trans_f[2],fp_fit,log2.(Narr),color=Makie.wong_colors()[1],label = "FP (m-b)")
#lines!(trans_f[2],mfit,log2.(Narr),color=Makie.wong_colors()[1],linestyle=:dash,label = "mean ")
lines!(trans_f[2],fp_ref_fit,log2.(Narr),color=Makie.wong_colors()[2],label = "FP (avg)")
#lines!(trans_f[2],mfit_ref,log2.(Narr),color=Makie.wong_colors()[2],linestyle=:dash,label = "mean ")

lines!(trans_V[2],fp_var,log2.(Narr),color=Makie.wong_colors()[1])
#lines!(trans_V[2],mvar,log2.(Narr),color=Makie.wong_colors()[1],linestyle=:dash)
lines!(trans_V[2],fp_ref_var,log2.(Narr),color=Makie.wong_colors()[2])
#lines!(trans_V[2],mvar_ref,log2.(Narr),color=Makie.wong_colors()[2],linestyle=:dash)


Colorbar(g2f[1,3],cb1,label = "log count")
Colorbar(g2V[1,3],cb2, label = "log count")

trans_f[1].ylabel = L"\log_2 N"
trans_V[1].ylabel = L"\log_2 N"
for i in 1:2
    trans_f[i].xlabel = L"\bar{f}(z) - f_c"
    trans_V[i].xlabel = L"\bar{V}(z) - V_c"
end

trans_f[1].title = "mini-batching"
trans_V[1].title = "mini-batching"
trans_f[2].title = "average"
trans_V[2].title = "average"

axislegend(trans_f[2],position=:lt, framevisible = false,patchsize=(15,10))

for (label, layout) in zip(["A", "B"], [gde, g2])
    Label(layout[1, 1, TopLeft()], label,
        fontsize = 20,
        font = :bold,
        padding = (0, 5, 5, 0),
        halign = :right)
end

rowsize!(f.layout, 1, Auto(2.4))

save("fig3spec_5.pdf",f)

f  